In [1]:
import os
import shutil



import tensorflow as tf

# 1. 훈련 데이터 복사하기
train 폴더에서 cat, dog 폴더를 만들어 해당 사진을 이용시키고<br>
훈련데이터와 검증데이터를 나눔

In [2]:
original_dataset_path = './dataset'
copy_train_path = './datasets'

In [3]:
def copy_files(dogORcat_path, start_num, end_num, trainORval_path):
    
    image_paths = [os.path.join(original_dataset_path, 'train', dogORcat_path +'.'+str(i)+'.jpg')
                  for i in range(start_num, end_num)]
    
    target_copy_paths = os.path.join(copy_train_path, trainORval_path, dogORcat_path)
    
    if not os.path.isdir(target_copy_paths):
        os.makedirs(target_copy_paths)
        
    for image_path in image_paths:
        shutil.copy(image_path, target_copy_paths)
        
    print('데이터 복사가 완료되었습니다.')
    

In [4]:
copy_files('dog',0,10000,'train')
copy_files('cat',0,10000,'train')
copy_files('dog',10000,12500,'validation')
copy_files('cat',10000,12500,'validation')

데이터 복사가 완료되었습니다.
데이터 복사가 완료되었습니다.
데이터 복사가 완료되었습니다.
데이터 복사가 완료되었습니다.


In [5]:
print('훈련데이터 dog 개수', len(os.listdir('./datasets/train/dog')))
print('훈련데이터 cat 개수', len(os.listdir('./datasets/train/cat')))
print('검증데이터 dog 개수', len(os.listdir('./datasets/validation/dog')))
print('검증데이터 cat 개수', len(os.listdir('./datasets/validation/cat')))

훈련데이터 dog 개수 10000
훈련데이터 cat 개수 10000
검증데이터 dog 개수 2500
검증데이터 cat 개수 2500


# 콜백 함수 만들기
## 콜백 함수를 만드는 이유
 - 모델 훈련시 통제를 할 수 있기 때문
 - 콜백 함수가 없으면 좋던 싫든 훈련이 끝날 때까지 아무것도 제어 할 수가 없다.
 - 콜백 함수는 베스트 가중치 선별 저장부터 모델 부적격시 자동 중단할 수 있다.
 - 다양한 기능이 있으니 관련 자료 찾아 보고 원하는 셋팅

In [6]:
from datetime import datetime
import keras

Using TensorFlow backend.


In [7]:
save_dir = './my_log'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
project_name = 'dog_cat_CNN_model'

In [8]:
def save_file():
    time = datetime.today()
    yy = time.year
    mon = time.month
    dd = time.day
    hh = time.hour
    mm = time.minute
    sec = time.second
    time_name = str(yy) + str(mon) + str(hh) + str(mm) + str(sec) + 'my_' + project_name + '_model.h5'
    file_name = os.path.join(save_dir, time_name)
    return file_name

In [9]:
callback = [
    
#     keras.callbacks.TensorBoard(
#     log_dir = save_dir, #텐션보드 그래프를 저장할 디렉토리를 지정
#     write_graph = True, #graph 사용
#     write_images = True #image 사용
#     ),
    
    keras.callbacks.EarlyStopping(  # 학습중 평가시 모델 훈련 종료 시점을 지정
    monitor = 'val_acc',            # 종료 시키기 위한 관찰 대상
        patience = 10               # 모니터의 대상의 평가가 지정한 횟수 만큼 나아지지 않으면 훈련을 종료
    ),                    #ex) monitor = 'val_acc', patience =5 이라면
                            # 검증데이터 정확도가 5에포크만큼 나아지지 않으면 훈련을 종료합니다.
    keras.callbacks.ModelCheckpoint(
    filepath = save_file(),    # 모델 저장 위치
    monitor = 'val_loss',      # 모델에 저장할 가중치, 주로 'val_loss' 사용
    save_best_only = True      # 최상의 가중치만 기록여부
    ),
    
#     keras.callbacks.ReduceLROnPlateau(
#         monitor='val_acc',
#         patience = 10,
#         verbose = 1,
#         factor =0.5,
#         min_lr=0.00001)

]

# 3. 모델 구성

In [10]:
from keras import Input
from keras import layers, models, losses, optimizers

In [11]:
batch_size = 256
no_classes = 1
epochs = 32
image_height, image_width = 28,28     # 원래는 150,150
input_shape = (image_height, image_width, 3)

In [12]:
def cnn_api(input_shape):
    
    input_tensor = Input(input_shape, name = 'input')
    
    x = layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu')(input_tensor)
    x = layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units = 1024, activation = 'relu')(x)
    
    output_tensor = layers.Dense(units = no_classes, activation = 'sigmoid', name = 'output')(x)
    
    model = models.Model([input_tensor], [output_tensor])
    
    model.compile(loss = losses.binary_crossentropy, optimizer = optimizers.RMSprop(lr = 0.0001), metrics = ['acc'])
    
    return model
                                                                                                                

In [13]:
cnn_api(input_shape).summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
output (Dense)               (None, 1)                 1025

# 4. 데이터 전처리 

In [14]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(copy_train_path, 'train'),
    target_size = (image_height, image_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(copy_train_path, 'validation'),
    target_size = (image_height, image_width),
    batch_size = batch_size,
    class_mode= 'binary'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
newType_model = cnn_api(input_shape)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
output (Dense)               (None, 1)                 1025

In [ ]:
newType_model = cnn_api(input_shape)
newType_model
hist = newType_model.fit_generator(
    train_generator,
    steps_per_epoch = 20000//batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = 5000//batch_size,
    callbacks = callback)

In [ ]:
import matplotlib.pyplot as plt
train_acc = hist.history['acc']
val_acc = hist.history['val_acc']

train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1,len(train_acc)+1)

plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Val acc')
plt.title('Training and Val loss')
plt.legend()

plt.figure()
plt.plot(epochs, train_loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label= 'Val loss')
plt.show()

# 텐셔보드 

# 11. 과대 적합